In [178]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [194]:
class Value:
    def __init__(self, data, _children=(), _op = '', label = ''):
        self.data = data
        self._prev = set(_children)
        self._backward = lambda: None
        self._op = _op
        self.label = label
        self.grad = 0.0

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other

    def exp(self):
        out = Value(math.exp(self.data), (self, ), 'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other ** -1

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting inf/float for now"
        out = Value(self.data**other, (self, ), f"**{other}")

        def _backward():
            self.grad = other * self.data ** (other - 1) * out.grad

        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x)- 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0

        for node in reversed(topo):
            node._backward()

In [195]:
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

b = Value(6.8813735870195432, label='b')

x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label ='x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label="n"
o = n.tanh(); o.label="o"

In [300]:
import random


class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        act = sum(wi*xi for wi, xi in zip(self.w, x)) + self.b
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [337]:
n = MLP(3, [16,16,1])
xs = [
    [2.0, 3.0, -1.0], 
    [1.0, -1.0, -1.0], 
    [2.0, 2.0, 2.0], 
    [-2.0, 3.0, 1.0], 
]
ys = [1.0, -0.5, 1.0, 1.0]


In [338]:
for i in range(100):
    ypred = [n(x) for x in xs]
    loss = sum([(yout-ygt)**2 for ygt, yout in zip(ys, ypred)])

    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    for p in n.parameters():
        p.data += -0.05 * p.grad

    print(i, loss.data)

0 7.332075594686841
1 0.0008633270396760633
2 0.0009389433864857672
3 0.0006818174837543947
4 0.0007272042058839925
5 0.0005432974630756169
6 0.0005697072502469121
7 0.0004357129196221314
8 0.00045012352288885046
9 0.0003510943059416954
10 0.0003579423859332514
11 0.00028391851414832426
12 0.00028606810223428495
13 0.00023021822649659575
14 0.00022953184475325676
15 0.000187063650615124
16 0.0001847531623461092
17 0.00015224391276186326
18 0.0001490928301569016
19 0.00012406206770383186
20 0.00012056973173352892
21 0.00010119805261697165
22 9.76747658433434e-05
23 8.26140264346918e-05
24 7.924459445970548e-05
25 6.748713652585483e-05
26 6.437364684969039e-05
27 5.5160612751015116e-05
28 5.2351354173376954e-05
29 4.510746110619377e-05
30 4.2616483798657135e-05
31 3.690304053524267e-05
32 3.472334412977319e-05
33 3.0204044510009584e-05
34 2.8316406193936554e-05
35 2.4732192778774253e-05
36 2.3111005741947787e-05
37 2.0261447807002608e-05
38 1.8878511136306808e-05
39 1.6607909049533357e-0

In [339]:
ypred

[Value(data=0.9995030982788619),
 Value(data=-0.49980420034146944),
 Value(data=0.9999664434036881),
 Value(data=0.9999488230322644)]